In [2]:
import pandas as pd
import numpy as np
import pickle


In [3]:
df=pd.read_csv('/home/rahulpuppala/Documents/Major Project 2/artifacts/cleaned_data.csv')
df.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.50,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [4]:
with open("/home/rahulpuppala/Documents/Major Project 2/artifacts/preprocessor.pkl",'rb') as file:
    preprocessor=pickle.load(file)

In [5]:
train_df=pd.read_csv("/home/rahulpuppala/Documents/Major Project 2/artifacts/train.csv")
test_df=pd.read_csv("/home/rahulpuppala/Documents/Major Project 2/artifacts/test.csv")


In [6]:
train_df.drop(columns=['Unnamed: 0'],inplace=True)
test_df.drop(columns=['Unnamed: 0'],inplace=True)

In [7]:
from tensorflow.keras.utils import to_categorical
target_feature="Churn"

input_feature_train_df=train_df.drop(columns=[target_feature],axis=1)
y_train=train_df[target_feature].replace({'Yes':1,'No':0}).astype(int)

input_feature_test_df=test_df.drop(columns=[target_feature],axis=1)
y_test=test_df[target_feature].replace({'Yes':1,'No':0}).astype(int)


# y_train = to_categorical(y_train, num_classes=17)
# y_test = to_categorical(y_test, num_classes=17)

X_train=preprocessor.fit_transform(input_feature_train_df)
X_test=preprocessor.transform(input_feature_test_df)


2025-04-06 19:47:46.241562: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-06 19:47:46.266492: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-06 19:47:46.388586: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-06 19:47:46.498572: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743949066.612446    6845 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743949066.63

In [8]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [9]:
model=Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64,activation='relu'),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid') # output layer
]
)

2025-04-06 19:47:49.717964: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         2,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,993 (19.50 KB)

 Trainable params: 4,993 (19.50 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
opt=tf.keras.optimizers.Adam(learning_rate=0.01)
loss=tf.keras.losses.BinaryCrossentropy()

In [12]:
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])

In [13]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [ ]:
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)

In [25]:
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.7795 - loss: 0.4658 - val_accuracy: 0.7757 - val_loss: 0.5045
Epoch 2/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8076 - loss: 0.4322 - val_accuracy: 0.7885 - val_loss: 0.4387
Epoch 3/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8105 - loss: 0.4203 - val_accuracy: 0.8070 - val_loss: 0.4155
Epoch 4/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7964 - loss: 0.4254 - val_accuracy: 0.8020 - val_loss: 0.4190
Epoch 5/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8022 - loss: 0.4118 - val_accuracy: 0.7977 - val_loss: 0.4316
Epoch 6/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8106 - loss: 0.4074 - val_accuracy: 0.8048 - val_loss: 0.4158
Epoch 7/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7998 - loss: 0.4253 - val_accuracy: 0.7913 - val_loss: 0.4185
Epoch 8/100
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8165 - loss: 0.4048 - val_accu

In [28]:
model.evaluate(X_test, y_test)

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8051 - loss: 0.4301


[0.4154743254184723, 0.8069552779197693]

In [29]:
y_pred = model.predict(X_test)
y_pred_class = (y_pred >= 0.5).astype(int)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_class))


45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
              precision    recall  f1-score   support

           0       0.83      0.93      0.87      1027
           1       0.71      0.49      0.58       382

    accuracy                           0.81      1409
   macro avg       0.77      0.71      0.73      1409
weighted avg       0.80      0.81      0.79      1409



From the above observation, we can say that the accuracy is 80% using ann

In [15]:
# model prediction
data={
    'gender':'male',
    'SeniorCitizen':0,
    'Partner':'No',
    'Dependents':'No',
    'tenure':'9',
    'PhoneService':'No',
    'MultipleLines': 'No',
    'InternetService':'No',
    'OnlineSecurity':'No',
    'OnlineBackup':'Yes',	
    'DeviceProtection':'No',
    'TechSupport':'No',
    'StreamingTV':'Yes',
    'StreamingMovies':'Yes',
    'Contract':'One Year',
    'PaperlessBilling':'Yes',
    'PaymentMethod': 'Electronic check',
    'MonthlyCharges':111.3,
    'TotalCharges': 5450.7

}

In [16]:
new_data=pd.DataFrame(data,index=[0])

In [17]:
new_data

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,male,0,No,No,9,No,No,No,No,Yes,No,No,Yes,Yes,One Year,Yes,Electronic check,111.3,5450.7


In [18]:
new_data=preprocessor.transform(new_data)

In [19]:
new_data

array([[-0.94328209,  1.55056523,  1.40408367,  0.        ,  0.        ,
         1.        ,  0.        ,  1.        ,  0.        ,  1.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  1.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  1.        ,  1.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  1.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  1.        ,
         0.        ,  0.        ,  1.        ,  0.        ]])

In [20]:
predict_churn=model.predict(new_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step


In [21]:
predict_churn

array([[0.5301729]], dtype=float32)

In [22]:
if predict_churn > 0.5:
    print("the above customer is likely to exit your services.")
else:
    print("The above customer is going to continue his services")

the above customer is likely to exit your services.


In [23]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │         2,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,993 (19.50 KB)

 Trainable params: 4,993 (19.50 KB)

 Non-trainable params: 0 (0.00 B)